In [1]:
import pandas as pd
import numpy as np

In [48]:
dataset = pd.read_csv('data/employees_dataset.csv')
#skills和work_experience列都是多给字符串用分号分隔，此处我想把这两列分别转换成技能个数和经验个数
dataset.head(10)

,degree,education,skills,working_experience,position
0,master,fudan university,c/c++ stl;c;sql;design patterns;atl com;asp.ne...,bank of china;sap;emc,dev
1,bachelor,inner mongolia university,c;design patterns;windows;system modeling;stor...,ca technologies;beijing topsec network securit...,dev
2,bachelor,shanghai university,c;git;java;python;mongodb;selenium;test automa...,tudou.com;ebaotech corporation,dev
3,master,shanghai jiao tong university,c;java;c++;provisioning;unix;clariion;solaris;...,hp;emc,dev
4,master,beijing university of post and telecommunications,c;sip;java;tcp/ip;shell scripting;python;ip;cl...,tektronix,dev
5,master,dalian university of technology,c;sip;shell scripting;operating systems;device...,synopsys;alcatel-lucent,dev
6,master,beihang university,c++/c;java;sql;python;design patterns;generic ...,emc information technology r&d co. ltd.;ibm,dev
7,master,university of california berkeley - walter a....,cloud applications;web services;big data;cloud...,yahoo;mywire;augmentum,dev
8,master,nanjing university,customer communication;cloud computing;trouble...,ibm;emc,dev
9,master,beijing institute of technology,debugging;c;assembly;usb;kernel;device drivers...,innofidei;symantec;intel,dev


In [57]:
#对于其他列使用把不同类别映射成不同数字的方式
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y=le.fit_transform(dataset['position'].values)
used_features = ["degree", "education"]
X = dataset[used_features].values

In [58]:
X[:,0]=le.fit_transform(X[:,0])
X[:,1]=le.fit_transform(X[:,1])

In [65]:
def word2Vector(data):
    temp=[]
    words=[];
    rs=[]
    for i in data:
        split = i.split(';')
        for i in split:
            words.append(i)
        temp.append(split)
    words = list(set(words))
    for i in temp:
        tt=[]
        for j in words:
            if j in i:
                tt.append(0)
            else:
                tt.append(1)
        rs.append(tt)
    return rs

In [71]:
X=np.concatenate((X, word2Vector(dataset['skills'].values)), axis=1)
X=np.concatenate((X, word2Vector(dataset['working_experience'].values)), axis=1)


In [72]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 0)# 按分割测试数据与训练数据

In [73]:
#训练模型
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train,Y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [74]:
y_pred = gnb.predict(X_test)#预测测试数据

In [75]:
print("Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%"
          .format(
              X_test.shape[0],
              (Y_test != y_pred).sum(),
              100*(1-(Y_test != y_pred).sum()/X_test.shape[0])
    ))

Number of mislabeled points out of a total 8 points : 2, performance 75.00%


In [76]:
from sklearn.linear_model import LogisticRegression 
lr = LogisticRegression(C=1e5)  
lr.fit(X_train,Y_train)
y_pred = lr.predict(X_test)#预测测试数据
print("Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%"
          .format(
              X_test.shape[0],
              (Y_test != y_pred).sum(),
              100*(1-(Y_test != y_pred).sum()/X_test.shape[0])
    ))

Number of mislabeled points out of a total 8 points : 0, performance 100.00%


c:\users\thinking\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\thinking\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [77]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(X_train,Y_train)
y_pred = clf.predict(X_test)#预测测试数据
print("Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%"
          .format(
              X_test.shape[0],
              (Y_test != y_pred).sum(),
              100*(1-(Y_test != y_pred).sum()/X_test.shape[0])
    ))

Number of mislabeled points out of a total 8 points : 0, performance 100.00%
